## Enable managed identity on existing Storage Account and Cosmos DB Account 

### Login to the subscription

In [ ]:
az login
az cloud set --name AzureCloud
az account set --subscription $subscription
if  ($(az group exists --name $rg) -eq $false) { az group create --name $rg --location $location }
az config set defaults.group=$rg

### Provide Sub Id, Resource Group, Cosmos DB Account Name, Storage Account and Region

- please note that armlocation is required for private preview az cli extension, _please don't modify it_

In [ ]:
$subscription = "subId"
$rg = "rgName"
$cosmosdbaccountname = "cosmosDbAccountName"
$stgrg = "stgRgName"
$storageaccount = "stgAccName" #only lowercase letters and numbers
$location = "region"
$armlocation = "centraluseuap"

### Load Data Transfer Private AZ CLI Package
please ignore below warning and error in the output(this is common if you re-execute the script):

- WARNING: Command group 'config' is experimental and under development
    
- ERROR: The extension cosmosdb-preview already exists

In [ ]:
az config set defaults.group=$rg
az cloud update --endpoint-resource-manager "https://$armlocation.management.azure.com"
az config set defaults.location=$location --local
az extension add --source https://dtsclidist.blob.core.windows.net/cli-private-preview/V2/cosmosdb_preview-0.10.0-py2.py3-none-any.whl -y

### Assign Cosmos DB Managed System Identity to Storage Account
- please note that if the role already exists the output shows 'Azure Error: RoleAssignmentExists', it's safe to ignore this error.

In [ ]:
az cosmosdb identity assign -n $cosmosdbaccountname -g $rg
az cosmosdb update -n $cosmosdbaccountname --default-identity="SystemAssignedIdentity"
$identityPrincipal=$(az cosmosdb identity show -n $cosmosdbaccountname -g $rg --query principalId --out tsv)
az role assignment create --assignee $identityPrincipal --role "Storage Blob Data Contributor" --scope "subscriptions/$subscription/resourceGroups/$stgrg/providers/Microsoft.Storage/storageAccounts/$storageaccount"